In [1]:
import os, sys
import pyproj
from pyproj import Proj 
from netCDF4 import Dataset
import netCDF4, h5netcdf, zarr
import xarray as xr
import shutil, glob
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import datetime
import cftime
import subprocess
import shutil
from functools import reduce
from config import Config, Constant
import logging
from cdo import Cdo
import warnings
warnings.filterwarnings('ignore')

#### Concatenation des fichiers netcdf

In [2]:
def concat_nc_files(folder_name, output_filename="output.nc"):
    # Liste pour stocker les datasets
    datasets = []
    
    # Parcours des fichiers dans le dossier
    for filename in os.listdir(folder_name):
        if filename.endswith(".nc"):

            #change_missing_val(filename,folder_name)
            # Chargement du fichier NetCDF
            ds = xr.open_dataset(os.path.join(folder_name, filename), engine='netcdf4')
            datasets.append(ds) 

    # Concaténation des datasets
    combined = xr.concat(datasets, dim="time")
    
    # Sauvegarde dans un fichier NetCDF
    combined.to_netcdf(output_filename)

#### Changement des valeurs manquantes des fichiers netcdf

In [3]:
def change_missing_val(folder_in, folder_out):
    # Construire le chemin complet du fichier
    os.makedirs(folder_out, exist_ok=True)

    for filename in os.listdir(folder_in):
        if filename.endswith(".nc"):

            full_input_path = os.path.join(folder_in, filename)
            full_output_path=os.path.join(folder_out,filename)

            # Changer les valeurs manquantes avec CDO
            change = f'cdo setmissval,nan {full_input_path} {full_output_path}'
            subprocess.run(change, shell=True)


In [ ]:
def calculate_rolling_average(df_data, num_months):
    df=df_data.copy()
    # Triez le DataFrame par 'Station', 'Year' et 'Month'
    df = df.sort_values(['Station', 'Year', 'Month'])
    columns_to_average=['v_wind_925','u_wind_850', 'u_wind_700', 'u_wind_200', 'eau_precipitable', 't_point_rosee', 'h_vol_sol_wat', 'anom_lef_mois', 'anom_nino_mois']
    # Utilisez la méthode rolling() pour calculer la moyenne roulante sur le nombre de mois spécifié
    rolling_average = df.groupby('Station')[columns_to_average].rolling(window=num_months).mean()
    # Réinitialisez l'index
    rolling_average = rolling_average.reset_index()
    rolling_average = rolling_average.set_index('level_1')
    for col in columns_to_average:
        df[col] = rolling_average[col]
    return df

In [4]:
#Lecture du fichier des données des coordonnées des stations
#Ce fichier permet d'extraire les valeurs au dessus des stations à partir des fichiers NetCDF
station_infos_file=os.path.join(Config.STATION_PATH,Config.STAT_COORD_FILE_NAME)

stations = pd.read_excel(station_infos_file)

In [5]:
# Liste contenant les noms des dataframes decades et mensuels
df_terre_dek=[]
df_terre_mon=[]
df_mer_dek=[]
df_mer_mon=[]

In [6]:
# Variables de configuration
folder_original = Config.V_WIND_925_DEK
wind_925_v = Config.WIND_V_925
# Repertoire sortie des données avec missing values corrigées
folder_miss_check=Config.V_WIND_925_VAL
# Chemin de sortie
output_path = os.path.join(Config.ERA5_PATH, wind_925_v)
#concat_nc_files(folder_name, output_path)

change_missing_val(folder_original,folder_miss_check)

In [7]:
concat_nc_files(folder_miss_check, output_path)

In [ ]:
# Lecture de fichier de vent v à la 925hPa
v_wind_925=Dataset(output_path)
#Lecture des variables du fichier au format netCDF
v_wind=v_wind_925.variables['v'][:]
unitd=v_wind_925.variables['v'].units
unit=v_wind_925.variables['time'].units
times=v_wind_925.variables['time'][:]
lat=v_wind_925.variables['latitude'][:]
lon=v_wind_925.variables['longitude'][:]

In [ ]:
print(v_wind_925)

In [ ]:
val_miss=Dataset('V_WIND_925_1961_01_01_OK.nc')
for var in val_miss.variables.values():
    print(var)

In [ ]:
val=Dataset('V_WIND_925_1961_01_01.nc')
for var in val.variables.values():
    print(var)

In [ ]:
val_miss = xr.open_dataset('V_WIND_925_1961_01_01_OK.nc')

# Afficher les informations du dataset
print(val_miss)

# Parcourir les variables pour trouver les attributs spécifiques des valeurs manquantes
for var in val_miss.data_vars:
    print(f"Variable: {var}")
    print(val_miss[var].attrs)

In [ ]:
variable_name = "v"  # Remplacez par le nom de votre variable
variable_data = val_miss[variable_name]

# Afficher les attributs de la variable
print(variable_data.attrs)

# Afficher la valeur manquante
missing_value = variable_data.attrs.get('_FillValue', None)
if missing_value is None:
    missing_value = variable_data.attrs.get('missing_value', None)

print(f"La valeur manquante pour {variable_name} est : {missing_value}")


In [ ]:
val_miss = xr.open_dataset('V_WIND_925_1961_01_01.nc')

# Afficher les informations du dataset
print(val_miss)

# Parcourir les variables pour trouver les attributs spécifiques des valeurs manquantes
for var in val_miss.data_vars:
    print(f"Variable: {var}")
    print(val_miss[var].attrs)

In [ ]:
variable_name = "v"  # Remplacez par le nom de votre variable
variable_data = val_miss[variable_name]

# Afficher les attributs de la variable
print(variable_data.attrs)

# Afficher la valeur manquante
missing_value = variable_data.attrs.get('_FillValue', None)
if missing_value is None:
    missing_value = variable_data.attrs.get('missing_value', None)

print(f"La valeur manquante pour {variable_name} est : {missing_value}")